In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Training, validation and testing of the MLP-SI model
#####################################################################################

In [ ]:
#######################################################################################################
# Baseline MLP (assuming independence between timesteps)
# Subject-independent
# Training set is shuffled (by Keras)
#######################################################################################################
# VALIDATION / ARCHITECTURE TUNING
# DONE
#######################################################################################################
import numpy as np
import time
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from geoutils import radToDeg, degToRad
from evalutils import norm_Y, inv_norm_Y, eval_test, plot_predictions
from settings import *

AF_type = 'AF_logFB26_norm'
AF_type = 'AF_logFB52_norm' # SET tuning_type
AF_type = 'AF_logFB78_norm'
# AF_type = 'AF_MFCC13_norm'

TE_folder = 'TrainingExamples_16kHz'

# XXX prefix means NO DROPOUT
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXX'

FPS = 100.

#######################################################################################################
np.random.seed(37) # for reproducibility

unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..

AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
N_features = AF.shape[1]
PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]
N_targets = PF.shape[1]

#######################
# Target (Y) normalisation, into range 0-1 according to constraints
PF = norm_Y(PF)
print "Targets (Y) are TRANSFORMED to 0-1 range"

#######################
# Load the dataset split  
ds = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_all.npz')
train_mask = ds['train_mask']
val_mask   = ds['val_mask']
test_mask  = ds['test_mask']

X_train = AF[train_mask]
X_val   = AF[val_mask]
X_test  = AF[test_mask]
del AF
Y_train = PF[train_mask]
Y_val   = PF[val_mask]
Y_test  = PF[test_mask]
del PF

#######################
# Learning settings
epochs = 1000
N_runs = 1
dropout = 0.

# Architectures to try
N_hl_range = [1, 2, 3, 5, 7] # range of numbers of hidden layers
N_hu_range = [8, 16, 32, 64, 128, 256, 512] # range of numbers of units per hidden layer

print "Validation over ", len(N_hl_range) * len(N_hu_range), "=", len(N_hl_range), "x", len(N_hu_range), "parameter settings"

tuning_type = str(N_runs) + '_' + str(len(N_hl_range)*len(N_hu_range)) + '_AF' + AF_type.split('_')[1][-2:]

print TE_folder, AF_type, tuning_type
if tuning_type[-2:] != AF_type.split('_')[1][-2:]:
    raise ValueError("Tuning type and audio feature type mismatch!")

#######################
# Validation
train_batch_size = len(X_train)
val_batch_size = len(X_val)
test_batch_size = len(X_test)

vals = np.zeros((len(N_hl_range), len(N_hu_range)))
vals_std = np.zeros((len(N_hl_range), len(N_hu_range)))

st = time.time()
for a, N_hl in enumerate(N_hl_range):
    for b, N_hu in enumerate(N_hu_range):
        print "HL, HU: ", N_hl, N_hu
        #st = time.time()
        # Create model
        model = Sequential()
        model.add(Dense(N_hu, activation='relu', kernel_initializer='he_uniform', input_dim=N_features))
        #model.add(Dropout(dropout))
        for i in range(1, N_hl):
            model.add(Dense(N_hu, activation='relu', kernel_initializer='he_uniform'))
            #model.add(Dropout(dropout))
        model.add(Dense(N_targets, activation='sigmoid'))

        model.compile(loss='mean_squared_error', optimizer=Adam())
        #print model.summary()
        early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1) # stop after 10 epochs without improvement in val_loss

        vals_actual = []
        for i in range(N_runs):
            model.fit(X_train, Y_train, epochs=epochs, batch_size=train_batch_size, 
                       validation_data = (X_val, Y_val), verbose=0, callbacks=[early_stop])

            vals_actual.append( model.evaluate(X_val, Y_val, batch_size=val_batch_size, verbose=0) )

        vals[a][b] = np.mean(vals_actual)
        vals_std[a][b] = np.std(vals_actual)
        print "\tTime taken: ", time.time()-st, (time.time()-st)/60. 

best_N_hl = N_hl_range[np.argmin(vals) // len(N_hu_range)]
best_N_hu = N_hu_range[np.argmin(vals) % len(N_hu_range)]

plt.figure()
plt.imshow(vals.T, cmap=cmap) # , vmax = 0.6 , cmap=cmap
plt.xticks(range(len(N_hl_range)), N_hl_range)
plt.yticks(range(len(N_hu_range)), N_hu_range)
plt.xlabel('# hidden layers')
plt.ylabel('# hidden units per leayer')
plt.colorbar(orientation="horizontal", fraction=0.027)
plt.tight_layout()
plt.show()

print "\tOptimal number of hidden layers / hidden units: ", best_N_hl, " / ", best_N_hu
print "\tBest validation MSE: ", np.min(vals), vals[np.argmin(vals) // len(N_hu_range), np.argmin(vals) % len(N_hu_range)], vals.shape, np.argmin(vals)

# Save results
np.savez(save_results_path_prefix + 'val_' + tuning_type + '.npz', 
        vals=vals, vals_std=vals_std, best_N_hl=best_N_hl, best_N_hu=best_N_hu, 
        N_hl_range=N_hl_range, N_hu_range=N_hu_range, N_runs=N_runs, dropout=dropout)

In [ ]:
#######################################################################################################
# Show validation results
#######################################################################################################
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from settings import *

# # tuning_type = '1_35'
# tuning_type = '1_18'

# # tuning_type = '1_35_AF52'
# tuning_type = '1_18_AF52'

# # tuning_type = '1_35_AF78'
# tuning_type = '1_18_AF78'

tuning_types = [
    '1_35_AF13',
    '1_35_AF26', 
    '1_35_AF52', 
    '1_35_AF78'
]

TE_folder = 'TrainingExamples_16kHz'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXX'

for tuning_type in tuning_types:
    print tuning_type

    dd = np.load(save_results_path_prefix + 'val_' + tuning_type + '.npz')
    vals = dd['vals']
    vals_std = dd['vals_std']

    best_N_hl = int(dd['best_N_hl'])
    best_N_hu = int(dd['best_N_hu'])
    N_hl_range = dd['N_hl_range']
    N_hu_range = dd['N_hu_range']

    print "Validation over ", len(N_hl_range) * len(N_hu_range), "=", len(N_hl_range), "x", len(N_hu_range), "parameter settings"
    
    plt.figure()
    plt.imshow(vals.T, cmap=cmap, 
               #norm=LogNorm(vmin=vals.min(), vmax=vals.max())
              ) # , vmax = 0.6 , cmap=cmap
    plt.xticks(range(len(N_hl_range)), N_hl_range)
    plt.yticks(range(len(N_hu_range)), N_hu_range)
    plt.xlabel('# hidden layers')
    plt.ylabel('# hidden units per leayer')
    plt.colorbar(orientation="vertical", fraction=0.049)
#     plt.colorbar(orientation="horizontal", fraction=0.027)
    plt.tight_layout()
    plt.show()
    #print vals.T

    print "\tOptimal number of hidden layers / hidden units: ", best_N_hl, " / ", best_N_hu
    print "\tBest validation MSE: ", np.min(vals), vals[np.argmin(vals) // len(N_hu_range), np.argmin(vals) % len(N_hu_range)], vals.shape, np.argmin(vals)
    print "==================================================================================="

In [ ]:
#######################################################################################################
# TRAIN (using best architecture) 
# & EVALUATE ON VALIDATION SET: COMPARE 4 FEATURE SETS
# & EVALUATE ON TESTING SET (for logFB26 only)
# Saving 4 models, for each feature set
# DONE
#######################################################################################################
import numpy as np
import time
import glob

from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

from geoutils import radToDeg, degToRad
from evalutils import get_global_cca, get_local_cca, eval_test, norm_Y, inv_norm_Y, plot_predictions

SEGMENT_LEN = 300 # for evaluation (local cca)

tuning_types = [
    '1_35_AF13',
    '1_35_AF26', 
    '1_35_AF52', 
    '1_35_AF78'
]
AF_types = [
    'AF_MFCC13_norm',
    'AF_logFB26_norm',
    'AF_logFB52_norm',
    'AF_logFB78_norm'
]

TE_folder = 'TrainingExamples_16kHz'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXX'
model_checkpoint_path_prefix = './ModelCheckpoints/MLP_SI/'

np.random.seed(37) # for reproducibility
unique_srt_VIDs = unique_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_unique_srt_VIDs.npz')['unique_srt_VIDs'] # sorted VIDs
all_srt_VIDs = np.load('./../Dataset/'+TE_folder+'/te_VIDs.npz')['VIDs']
unique_srt_SIDs = np.array([x[:5] for i, x in enumerate(unique_srt_VIDs) if i % 2 == 0]) # ['PID02', 'PID05', ..
PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]
N_targets = PF.shape[1]
###########
# Target (Y) normalisation, into range 0-1 according to constraints
PF = norm_Y(PF)
print "Targets (Y) are TRANSFORMED to 0-1 range"

#######################
# Load the dataset split  
ds = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_all.npz')
train_mask = ds['train_mask']
val_mask   = ds['val_mask']
val_VIDs  = ds['val_VIDs']
val_VIDs_ind_cnts = ds['val_VIDs_ind_cnts']
test_mask  = ds['test_mask']
test_VIDs  = ds['test_VIDs']
test_VIDs_ind_cnts = ds['test_VIDs_ind_cnts']

Y_train = PF[train_mask]
Y_val   = PF[val_mask]
Y_test  = PF[test_mask]

FPS = 100.

# Learning settings
epochs = 1000
dropout = 0.

st = time.time()

for tuning_type, AF_type in zip(tuning_types, AF_types):
    
    print "Saving BEST model at:", model_checkpoint_path_prefix + tuning_type + '/'

    print TE_folder, AF_type, tuning_type
    if tuning_type[-2:] != AF_type.split('_')[1][-2:]:
        raise ValueError("Tuning type and audio feature type mismatch!")

    #######################################################################################################
    AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
    N_features = AF.shape[1]

    X_train = AF[train_mask]
    X_val   = AF[val_mask]
    X_test  = AF[test_mask]
    del AF

    #######################
    # Load validation data
    dd = np.load(save_results_path_prefix + 'val_' + tuning_type + '.npz')
    best_N_hl = int(dd['best_N_hl'])
    best_N_hu = int(dd['best_N_hu'])    
    print "\tOptimal number of hidden layers / hidden units: ", best_N_hl, " / ", best_N_hu

    train_batch_size = len(X_train)
    val_batch_size = len(X_val)
    test_batch_size = len(X_test)

    ##########################
    # Final train & test

    # Create model
    model = Sequential()
    model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform', input_dim=N_features))
    #model.add(Dropout(dropout))
    for i in range(1, best_N_hl):
        model.add(Dense(best_N_hu, activation='relu', kernel_initializer='he_uniform'))
        #model.add(Dropout(dropout))
    model.add(Dense(N_targets, activation='sigmoid'))

    model.compile(loss='mean_squared_error', optimizer=Adam())
    print model.summary()
    print "#parameters: ", model.count_params()
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1) # stop after 10 epochs without improvement in val_acc

    # Checkpoint model weights and the model itself: at each epoch
    model_checkpoint_name = 'm_{epoch:04d}_{loss:.4f}_{val_loss:.4f}.hdf5'
    model_checkpoint = ModelCheckpoint(model_checkpoint_path_prefix + tuning_type + '/' + model_checkpoint_name, monitor='val_loss', 
                                       verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    hist = model.fit(X_train, Y_train, epochs=epochs, batch_size=train_batch_size, 
               validation_data = (X_val, Y_val), verbose=1, callbacks=[early_stop, model_checkpoint])

    print "\tTime taken: ", time.time()-st, (time.time()-st)/60. 

    ###############################################################################################################
    # Evaluate on validation set
    Y_val_pred = model.predict(X_val, batch_size=val_batch_size, verbose=1)

    # Save results: predictions will be saved in radians; for generation on robot
    # Raw and smoothed (low-pass 4Hz)
    from postprocessingutils import save_predictions_and_eval
    save_predictions_and_eval(save_results_path_prefix + 'MSvaltest_' + tuning_type, 
                     X_val, Y_val, Y_val_pred, 'MLP_SI', SEGMENT_LEN, val_VIDs, val_VIDs_ind_cnts, 
                             N_params=model.count_params(), N_epochs=len(hist.history['loss']))
    
    ###############################################################################################################
    # Evaluate on testing set: only for 1_35_AF26
    if tuning_type == '1_35_AF26':
        Y_test_pred = model.predict(X_test, batch_size=test_batch_size, verbose=1)

        # Save results: predictions will be saved in radians; for generation on robot
        # Raw and smoothed (low-pass 4Hz)
        from postprocessingutils import save_predictions_and_eval
        save_predictions_and_eval(save_results_path_prefix + 'MStest_' + tuning_type, 
                         X_test, Y_test, Y_test_pred, 'MLP_SI', SEGMENT_LEN, test_VIDs, test_VIDs_ind_cnts, 
                                 N_params=model.count_params(), N_epochs=len(hist.history['loss']))    
    
    print "\tTime taken: ", time.time()-st, (time.time()-st)/60. 

In [ ]:
#######################################################################################################
# Re-Evaluate on VALIDATION & TEST SET using BEST MODEL
# DONE
#######################################################################################################
import numpy as np
import time
import glob
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from geoutils import radToDeg, degToRad
from evalutils import norm_Y, inv_norm_Y, eval_test, plot_predictions
from settings import *

from keras.models import load_model

tuning_types = [
    '1_35_AF13',
    '1_35_AF26', 
    '1_35_AF52', 
    '1_35_AF78'
]
AF_types = [
    'AF_MFCC13_norm',
    'AF_logFB26_norm',
    'AF_logFB52_norm',
    'AF_logFB78_norm'
]
model_names = [
    'm_1000_0.0141_0.0183.hdf5', 
    'm_0360_0.0143_0.0184.hdf5',
    'm_0903_0.0139_0.0185.hdf5',
    'm_0667_0.0140_0.0182.hdf5'
]

TE_folder = 'TrainingExamples_16kHz'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXX'
model_checkpoint_path_prefix = './ModelCheckpoints/MLP_SI/'

PF = np.load('./../Dataset/'+TE_folder+'/te_PF_smooth_LPBF_4.0.npz')['PF_smooth_LPBF']
PF = PF[:, :11]
N_targets = PF.shape[1]
###########
# Target (Y) normalisation, into range 0-1 according to constraints
PF = norm_Y(PF)
print "Targets (Y) are TRANSFORMED to 0-1 range"

#######################
# Load the dataset split  
ds = np.load('./../Dataset/'+TE_folder+'/Dataset_split/split_masks_all.npz')
val_mask   = ds['val_mask']
val_VIDs  = ds['val_VIDs']
val_VIDs_ind_cnts = ds['val_VIDs_ind_cnts']
test_mask  = ds['test_mask']
test_VIDs  = ds['test_VIDs']
test_VIDs_ind_cnts = ds['test_VIDs_ind_cnts']

Y_val   = PF[val_mask]
Y_test  = PF[test_mask]
st = time.time()
for tuning_type, AF_type, model_name in zip(tuning_types, AF_types, model_names):
       
    print TE_folder, AF_type, tuning_type
    if tuning_type[-2:] != AF_type.split('_')[1][-2:]:
        raise ValueError("Tuning type and audio feature type mismatch!")

    #######################################################################################################
    AF = np.load('./../Dataset/'+TE_folder+'/te_'+AF_type+'.npz')[AF_type]
    N_features = AF.shape[1]
    X_val   = AF[val_mask]
    X_test  = AF[test_mask]
    del AF
    
    val_batch_size = len(X_val)
    test_batch_size = len(X_test)

    #######################
    # Load best model
    test_model_name = model_checkpoint_path_prefix + tuning_type + '/' + model_name
    print "Loading BEST model from:", test_model_name
    model = load_model(test_model_name)
    
    
    ###############################################################################################################
    # Evaluate on validation set
    Y_val_pred = model.predict(X_val, batch_size=val_batch_size, verbose=1)

    # Save results: predictions will be saved in radians; for generation on robot
    # Raw and smoothed (low-pass 4Hz)
    from postprocessingutils import save_predictions_and_eval
    save_predictions_and_eval(save_results_path_prefix + 'MSBMvaltest_' + tuning_type, 
                     X_val, Y_val, Y_val_pred, 'MLP_SI', SEGMENT_LEN, val_VIDs, val_VIDs_ind_cnts, 
                             N_params=model.count_params())
    
    ###############################################################################################################
    # Evaluate on testing set: only for 1_35_AF26
    if tuning_type == '1_35_AF26':
        Y_test_pred = model.predict(X_test, batch_size=test_batch_size, verbose=1)

        # Save results: predictions will be saved in radians; for generation on robot
        # Raw and smoothed (low-pass 4Hz)
        from postprocessingutils import save_predictions_and_eval
        save_predictions_and_eval(save_results_path_prefix + 'MSBMtest_' + tuning_type, 
                         X_test, Y_test, Y_test_pred, 'MLP_SI', SEGMENT_LEN, test_VIDs, test_VIDs_ind_cnts, 
                                 N_params=model.count_params())    
    
    print "\tTime taken: ", time.time()-st, (time.time()-st)/60. 

In [ ]:
#######################################################################################################
# Show results on VALIDATION SET
# DONE
#######################################################################################################

import numpy as np
import time
import glob
from evalutils import show_test_results, plot_predictions

tuning_types = [
    '1_35_AF13',
    '1_35_AF26', 
    '1_35_AF52', 
    '1_35_AF78'
]

model_names = [
    'm_1000_0.0141_0.0183.hdf5', 
    'm_0360_0.0143_0.0184.hdf5',
    'm_0903_0.0139_0.0185.hdf5',
    'm_0667_0.0140_0.0182.hdf5'
]

TE_folder = 'TrainingExamples_16kHz'
save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXXMSBM'  # best model

for tuning_type, model_name in zip(tuning_types, model_names):

    print tuning_type
    d = np.load(save_results_path_prefix + 'valtest_' + tuning_type + '.npz')
    
    ###############################################################################
    # Show testing results: for raw Y and smoothed Y
    #print "===========================================Raw=====================\n"
    #show_test_results(d['results_raw'])
    print "===========================================Smooth=====================\n"
    show_test_results(d['results_smooth'])
    print "================================================================\n"

    ###############################################################################
    # Plot predictions (post-smoothed and raw) against ground truths and audio 
    t_VID = 1 # test VID to show
    if t_VID >= len(d['Y_raw_list']):
        raise ValueError("Required test VID is out of bounds!")
    Y_true = d['Y_true_list'][t_VID]
    Y_raw = d['Y_raw_list'][t_VID]
    Y_smooth = d['Y_smooth_list'][t_VID]
    test_VID = d['test_VIDs'][t_VID]

    plot_predictions(Y_true, Y_raw, Y_smooth, 'MLP_SI', angles_to_show='all', 
                         plot_start=13.0, plot_length=3.0, input_mode='time', SD_offset=None, 
                         test_VID=test_VID)

In [ ]:
#######################################################################################################
# Show results on TESTING SET
# DONE
#######################################################################################################

import numpy as np
import time
import glob
from evalutils import show_test_results, plot_predictions

tuning_type = '1_35_AF26'

TE_folder = 'TrainingExamples_16kHz'

save_results_path_prefix = './../Dataset/'+TE_folder+'/Results/MLP_SI/XXXMSBM'  # best model

d = np.load(save_results_path_prefix + 'test_' + tuning_type + '.npz')

print tuning_type

###############################################################################
# Show testing results: for raw Y and smoothed Y
#print "===========================================Raw=====================\n"
#show_test_results(d['results_raw'])
print "===========================================Smooth=====================\n"
show_test_results(d['results_smooth'])
print "================================================================\n"

###############################################################################
# Plot predictions (post-smoothed and raw) against ground truths and audio 
t_VID = 2 # test VID to show
if t_VID >= len(d['Y_raw_list']):
    raise ValueError("Required test VID is out of bounds!")
Y_true = d['Y_true_list'][t_VID]
Y_raw = d['Y_raw_list'][t_VID]
Y_smooth = d['Y_smooth_list'][t_VID]
test_VID = d['test_VIDs'][t_VID]

plot_predictions(Y_true, Y_raw, Y_smooth, 'MLP_SI', angles_to_show='all', 
                     plot_start=10.0, plot_length=3.0, input_mode='time', SD_offset=None, 
                     test_VID=test_VID)